# Popular Data Science Questions

[Stack Exchange](https://stackexchange.com/) is an extremely popular Q & A platform for a variety of software, IT, math, and "pretty much any other technical" question you can think of. 

They are most well known for their software question and answer platform [Stack Overflow](https://stackoverflow.com/).

For this project we will be taking a look at some of the more popular questions on a less known Stack Exchange site: [Data Science Stack-Exchange](https://datascience.stackexchange.com/).

### Questions for Data Science Stack Exchange

 - What kind of questions are welcome
    - Questions about machine learning, algorithims, and any other relevant data science question.
 
 - How does the home screen subdivide their questions:
    - Active, Bountied, Hot, Week, and Month
 
 - They also divide the navigation bar into several different sections:

      1. Home 
      2. Questions 
      3. Tags 
      4. Users
      5. Unanswered

Luckily Stack Exchange provides an open source [Data Base](https://data.stackexchange.com/datascience/query/new) we can query to find useful information. 

#### Promising Tables:

 1. Posts
 2. Users 
 3. Votes
 4. Tags
 5. PostTypes


I created a query that collected these items from Stack Exchanges's website.

*SELECT Id, PostTypeId, CreationDate, Score, ViewCount,
       Tags, AnswerCount, FavoriteCount*

 *FROM posts*

 *WHERE PostTypeID = 1 OR PostTypeID = 2 
   AND CreationDate BETWEEN '2019-01-01' AND '2021-01-01';*

---


This queried all the selected columns and only included questions, answers, and recent posts.

**Specifically between 2019 and 2020.**

### Analysis:

Let's start by reading in the dataset that we downloaded and exploring the data frame.